# Lab 2 - Probabilities, Distributions, and Statistical Questions
## William Olsen, partners with Jennifer Vo

In this lab we will become more familiar with probabilites, distributions, and statistical questions

## Problem 1

For this problem lets consier that we are looking for gamma-ray sources.  These sources are being detected by a telescope pointed off into space.  In outer space we have the cosmic background radiation which will be the background distribution for our telescope which we are using to detect gamma-ray sources.  This cosmic background happens to be modeled by precisely the Poisson distribution.  Now we would like to ask the following question regarding our background gamma-ray sources:

> What is the probability that cosmic-ray background would have provided us with a measurement signal-like or more than our actual signal?

For this problem we will assume that in a single day the average number of background cosmic-rays is 5.2 while the average number of gamma-rays is emitted by our hypothetical source is 7.7.

### Part A

Let us first observe how the probability distribution of the background varies as we integrate (or sum) for more days.


### Part B



### Part C



### Part D


## Problem 2

## Problem 3

## Problem 4